# Outlier detection

In [1]:
# https://github.com/fnielsen/everything
from everything import *

In [2]:
# Read dataframe with features for companies
filename = expanduser('~/workspace/cvrminer/virksomheder-features.csv')
df = read_csv(filename, encoding='utf-8', index_col=0)

In [3]:
# Feature names
df.columns

Index([u'antal_penheder', u'branche_ansvarskode', u'nyeste_antal_ansatte',
       u'nyeste_virksomhedsform', u'reklamebeskyttet', u'sammensat_status',
       u'sidste_virksomhedsstatus', u'stiftelsesaar'],
      dtype='object')

In [4]:
# Functions for conversion to numerical dataframes
def to_dummies(df, column):
    datatype = df[column].dtypes
    if datatype in [int64, float64]:
        return df[[column]]
    elif datatype == bool:
        return df[[column]].astype(int)
    elif datatype == 'object':
        df_column = df[column].str.get_dummies()
        df_column.columns = [column + ":" + col for col in df_column.columns]
        return df_column
    else:
        raise ValueError('Unrecognized datatype for column {}'.format(column))
        
def dataframe_to_numerical(df):
    df_numerical = DataFrame(index=df.index)
    for column in df.columns:
        print(column)
        df_numerical = df_numerical.join(to_dummies(df, column))
    return df_numerical

In [5]:
# Numerical dataframe
dfn = dataframe_to_numerical(df)
dfn.shape

antal_penheder
branche_ansvarskode
nyeste_antal_ansatte
nyeste_virksomhedsform
reklamebeskyttet
sammensat_status
sidste_virksomhedsstatus
stiftelsesaar


(1521456, 98)

In [6]:
dfn.describe()

/usr/local/lib/python2.7/dist-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,antal_penheder,branche_ansvarskode:0,branche_ansvarskode:15,branche_ansvarskode:65,branche_ansvarskode:75,branche_ansvarskode:76,branche_ansvarskode:96,branche_ansvarskode:97,branche_ansvarskode:99,branche_ansvarskode:None,...,sidste_virksomhedsstatus:UNDER FRIVILLIG LIKVIDATION,sidste_virksomhedsstatus:UNDER KONKURS,sidste_virksomhedsstatus:UNDER REASSUMERING,sidste_virksomhedsstatus:UNDER REASSUMMERING,sidste_virksomhedsstatus:UNDER REASUMMATION,sidste_virksomhedsstatus:UNDER REASUMMERING,sidste_virksomhedsstatus:UNDER REKONSTRUKTION,sidste_virksomhedsstatus:UNDER TVANGSOPLØSNING,sidste_virksomhedsstatus:slettet,stiftelsesaar
count,1.521456e+06,1.521456e+06,1.521456e+06,1.521456e+06,1.521456e+06,1.521456e+06,1.521456e+06,1.521456e+06,1.521456e+06,1.521456e+06,...,1.521456e+06,1.521456e+06,1.521456e+06,1.521456e+06,1.521456e+06,1.521456e+06,1.521456e+06,1.521456e+06,1.521456e+06,1.517460e+06
mean,1.002027e+00,1.102891e-03,1.934989e-03,4.729023e-03,6.730395e-04,1.314530e-06,1.557718e-04,6.046839e-05,1.097633e-04,9.912327e-01,...,1.096976e-03,5.636706e-03,8.938806e-05,6.572651e-07,6.572651e-07,2.629061e-06,7.229917e-06,1.912642e-03,2.366155e-05,1.999948e+03
std,2.358024e+00,3.319149e-02,4.394594e-02,6.860512e-02,2.593428e-02,1.146529e-03,1.247989e-02,7.775910e-03,1.047623e-02,9.322232e-02,...,3.310246e-02,7.486613e-02,9.454106e-03,8.107189e-04,8.107189e-04,1.621436e-03,2.688842e-03,4.369193e-02,4.864257e-03,1.432302e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.197000e+03
25%,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN
50%,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN
75%,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN
max,1.323000e+03,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.016000e+03


In [7]:
# Preprocessing
imputer = Imputer()
scaler = StandardScaler(with_mean=False)
dfni = scaler.fit_transform(imputer.fit_transform(dfn))

In [8]:
# Outlier detection/novelty detection with K-means clustering
clusterer = MiniBatchKMeans(n_clusters=8, verbose=True)
clusterer.fit(dfni)

/usr/local/lib/python2.7/dist-packages/sklearn/cluster/k_means_.py:1279: DeprecationWarning: This function is deprecated. Please call randint(0, 1521455 + 1) instead
  0, n_samples - 1, init_size)
/usr/local/lib/python2.7/dist-packages/sklearn/cluster/k_means_.py:630: DeprecationWarning: This function is deprecated. Please call randint(0, 1521455 + 1) instead
  0, n_samples - 1, init_size)
/usr/local/lib/python2.7/dist-packages/sklearn/cluster/k_means_.py:630: DeprecationWarning: This function is deprecated. Please call randint(0, 1521455 + 1) instead
  0, n_samples - 1, init_size)
/usr/local/lib/python2.7/dist-packages/sklearn/cluster/k_means_.py:630: DeprecationWarning: This function is deprecated. Please call randint(0, 1521455 + 1) instead
  0, n_samples - 1, init_size)
/usr/local/lib/python2.7/dist-packages/sklearn/cluster/k_means_.py:1328: DeprecationWarning: This function is deprecated. Please call randint(0, 1521455 + 1) instead
  0, n_samples - 1, self.batch_size)
/usr/local/l

Init 1/3 with method: k-means++
Inertia for init 1/3: 17680.132072
Init 2/3 with method: k-means++
Inertia for init 2/3: 17822.679048
Init 3/3 with method: k-means++
Inertia for init 3/3: 17328.389567
Minibatch iteration 1/1521500: mean batch inertia: 57.735447, ewa inertia: 57.735447 
Minibatch iteration 2/1521500: mean batch inertia: 57.171227, ewa inertia: 57.735372 
Minibatch iteration 3/1521500: mean batch inertia: 41.496874, ewa inertia: 57.733238 
Minibatch iteration 4/1521500: mean batch inertia: 146.608975, ewa inertia: 57.744921 
Minibatch iteration 5/1521500: mean batch inertia: 40.094582, ewa inertia: 57.742601 
Minibatch iteration 6/1521500: mean batch inertia: 27.508770, ewa inertia: 57.738626 
Minibatch iteration 7/1521500: mean batch inertia: 74.640563, ewa inertia: 57.740848 
Minibatch iteration 8/1521500: mean batch inertia: 38.592726, ewa inertia: 57.738331 
Minibatch iteration 9/1521500: mean batch inertia: 42.478083, ewa inertia: 57.736325 
[MiniBatchKMeans] Reassi

MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10, n_clusters=8,
        n_init=3, random_state=None, reassignment_ratio=0.01, tol=0.0,
        verbose=True)

In [9]:
distances = sum((dfni - clusterer.cluster_centers_[clusterer.labels_, :]) ** 2, axis=1)
indices_clusterer = argsort(-distances)

In [10]:
dfn.iloc[indices_clusterer[:20], :]

,antal_penheder,branche_ansvarskode:0,branche_ansvarskode:15,branche_ansvarskode:65,branche_ansvarskode:75,branche_ansvarskode:76,branche_ansvarskode:96,branche_ansvarskode:97,branche_ansvarskode:99,branche_ansvarskode:None,...,sidste_virksomhedsstatus:UNDER FRIVILLIG LIKVIDATION,sidste_virksomhedsstatus:UNDER KONKURS,sidste_virksomhedsstatus:UNDER REASSUMERING,sidste_virksomhedsstatus:UNDER REASSUMMERING,sidste_virksomhedsstatus:UNDER REASUMMATION,sidste_virksomhedsstatus:UNDER REASUMMERING,sidste_virksomhedsstatus:UNDER REKONSTRUKTION,sidste_virksomhedsstatus:UNDER TVANGSOPLØSNING,sidste_virksomhedsstatus:slettet,stiftelsesaar
cvr_nummer,,,,,,,,,,,,,,,,,,,,,
15706538,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1992.0
20899301,1,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1998.0
21976415,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1999.0
35852492,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,2014.0
31086477,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,2007.0
36467223,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,2014.0
56799559,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1985.0
27255116,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,2003.0
29219508,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,2006.0


In [11]:
filename = expanduser('~/workspace/cvrminer/virksomheder-report.html')
with codecs.open(filename, 'w', encoding='utf-8') as f:
    f.write("""
<html>
  <head>
    <title>Virksomheder report</title>
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
  </head>
  <body>
    <h1>Virksomheder report</h1>
    
""")
    
    f.write("""<h2>Numerical features dataframe summary statistics</h2>""")
    f.write(dfn.describe().T.to_html())
    f.write("""<h2>Novelty from k-means</h2>""")
    f.write(dfn.iloc[indices_clusterer[:50], :].to_html(
            escape=False,
            formatters={'__index__': 
                        lambda idx: '<a href="http://datacvr.virk.dk/data/visenhed?enhedstype=virksomhed&id={}">{}</a>'.format(
                     idx, idx)}))
    f.write("""
  </body>
</html>""")

In [ ]:
# Investigate cluster model as a function of number of clusters
inertias = []
for n_clusters in range(1, 50):
    clusterer = MiniBatchKMeans(n_clusters=n_clusters, max_iter=200, max_no_improvement=30, n_init=10)
    clusterer.fit(dfni)
    inertias.append(clusterer.inertia_)

In [ ]:
plot(inertias)
show()

In [ ]:
# Outlier detection with One-class-SVM
one_class_svm = OneClassSVM()
one_class_svm.fit(dfni)

In [ ]:
decisions = one_class_svm.decision_function(dfni)

In [ ]:
indices = argsort(decisions, axis=0)[:, 0]

In [ ]:
dfn.iloc[indices[:30], :]

In [ ]:
help(dfn.iloc[indices_clusterer[:20], :].to_html)